In [113]:
# Initial imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

  # Importing required Keras modules
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
%matplotlib inline

In [114]:
def read_symbol_file(symbol_file):
    symbol_df = pd.read_csv(symbol_file)

    symbol_df = symbol_df.rename(columns={'Unnamed: 0': 'DataDate'})
    symbol_df = symbol_df.set_index(pd.to_datetime(symbol_df.DataDate, infer_datetime_format=True))
    symbol_df = symbol_df.drop(symbol_df.columns[0], axis=1)
    
    return symbol_df

In [115]:
def process_lstm_symbol_file(symbol_df):

    #Create new trading signals Df, Set index as datetime object and drop extraneous columns
    trading_signals_df = pd.DataFrame()

    #add daily change rates to increase the staionarity of dataset
    trading_signals_df['volume delta'] = symbol_df['Volume'].dropna().pct_change()
    trading_signals_df['bb std delta'] = symbol_df['bollinger_std'].dropna().pct_change()
    trading_signals_df['rvol delta'] = symbol_df['rvol'].dropna().pct_change()
    trading_signals_df['option rvol delta'] = symbol_df['Option rVol'].dropna().pct_change()

    #add daily returns as target
    trading_signals_df['daily returns'] = symbol_df['daily returns'].dropna()

    trading_signals_df= trading_signals_df.fillna(value = 0)
    trading_signals_df= trading_signals_df.replace([np.inf, -np.inf], 0.0)
    trading_signals_df
    
    return trading_signals_df

In [116]:
def get_window_data(symbol_signals_df, window_size, feature_col_number, target_col_number):
    """
    This function accepts the column number for the features (X) and the target (y).
    It chunks the data up with a rolling window of Xt - window to predict Xt.
    It returns two numpy arrays of X and y.
    """
    X = []
    y = []
    for i in range(len(symbol_signals_df) - window_size):
        features = symbol_signals_df.iloc[i : (i + window_size), feature_col_number]
        
        print(features)
            
        target = symbol_signals_df.iloc[(i + window_size), target_col_number]
        
        
        X.append(features)
        y.append(target)
        
    return np.array(X), np.array(y).reshape(-1, 1)

In [117]:
def lstm_split_scale_reshape(X_df, y_df, split_ratio):
    split_range = int(split_ratio * len(X_df))

    X_train = X_df[: split_range]
    X_test = X_df[split_range:]

    y_train = y_df[: split_range]
    y_test = y_df[split_range:]
    
    
    scaler = MinMaxScaler()
    scaler.fit(X)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    scaler.fit(y)
    y_train = scaler.transform(y_train)
    y_test = scaler.transform(y_test)
    
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
    
    return X_train, X_test, y_train, y_test, scaler

In [129]:
def run_lstm(number_units, dropout_fraction, epochs, X_train, X_test, y_train):
    
    # Define the LSTM RNN model.
    model = Sequential()

    # Layer 1
    model.add(LSTM(
        units=number_units,
        return_sequences=True,
        input_shape=(X_train.shape[1], 1),
        activation = 'relu')
    ) 
    model.add(Dropout(dropout_fraction))
    
    # Layer 2
    model.add(LSTM(units=number_units, return_sequences=True, activation = 'relu'))
    model.add(Dropout(dropout_fraction))
    
    # Layer 3
    model.add(LSTM(units=number_units, activation = 'relu'))
    model.add(Dropout(dropout_fraction))
    
    # Output layer
    #model.add(Dense(1))
    model.add(Dense(1, activation = 'linear'))
    
    # Compile the LSTM model
    model.compile(optimizer="adam", loss="mean_squared_error")
    
    # Show the model summary
    print(model.summary())
    
    # Train the model
    print(model.fit(X_train, y_train, epochs=epochs, shuffle=False, batch_size=1, verbose=1))
    
    # Make predictions using the testing data X_test
    predicted_df = model.predict(X_test)
    
    return predicted_df

In [119]:
def process_predicted_data(predicted_df, y_test, scaler, symbol_signals_df):
    
    #Recover the original, nonscaled prices
    predicted_prices = scaler.inverse_transform(predicted_df)
    real_prices = scaler.inverse_transform(y_test.reshape(-1,1)) 
    
    shift =7
    stocks = pd.DataFrame({
        "Actual": real_prices.ravel(),
        "Predicted": predicted_prices.ravel()
    }, index = symbol_signals_df.index[-len(real_prices)-shift:-shift ] )
    
    return stocks

In [120]:
def process_stock_data(stocks, shift):
    
    #convert stocks df into positive and negative signals
    stocks['Positive Actual signal'] = np.where(stocks['Actual'] > 0, 1, 0)
    stocks['Negative Actual signal'] = np.where(stocks['Actual'] < 0, -1, 0)

    stocks['Positive Predicted signal'] = np.where(stocks['Predicted'] > 0, 1, 0)
    stocks['Negative Predicted signal'] = np.where(stocks['Predicted'] < 0, -1, 0)

    #merge to create one column per signal, shifted back to reflect forward projection window
    
    stocks['Actual Signal'] = stocks['Positive Actual signal'] + stocks['Negative Actual signal']
    stocks['LSTM Predicted Signal'] = stocks['Positive Predicted signal'] + stocks['Negative Predicted signal']
    
    return stocks

In [145]:
#set path to Features CSV and read in CSV
symbol_file = "TSLA_results.csv"

symbol_df = read_symbol_file(symbol_file)

#symbol_df.info()

symbol_signals_df = process_lstm_symbol_file(symbol_df)

print(symbol_signals_df.head(15))

window_size = 7
feature_col_number = 1 
target_col_number = -1

# Create the features (X) and target (y) data using the window_data() function.
X, y = get_window_data(symbol_signals_df, window_size, feature_col_number, target_col_number)
print(X[:15])
print(y[:15])

split_ratio = 0.7
X_train, X_test, y_train, y_test, scaler = lstm_split_scale_reshape(X, y, split_ratio)


number_units = 50
dropout_fraction = 0.2
epochs = 50

predicted = run_lstm(number_units, dropout_fraction, epochs, X_train, X_test, y_train)
predicted

stocks_df = process_predicted_data(predicted, y_test, scaler, symbol_signals_df)

shift = 7

stocks_df = process_stock_data(stocks_df, shift)
    
stocks_df

            volume delta  bb std delta  rvol delta  option rvol delta  \
DataDate                                                                
2016-02-26      0.000000      0.000000    0.000000           0.000000   
2016-02-29     -0.258215     -0.612534   -0.267351          -0.395173   
2016-03-01      0.491932      0.259721    0.476312           0.471753   
2016-03-02     -0.275588     -0.016266   -0.270669          -0.198582   
2016-03-03     -0.006869      0.749635    0.016632           0.154893   
2016-03-04      0.343777      0.289201    0.322556           0.313045   
2016-03-07     -0.178715     -0.249929   -0.152581          -0.388585   
2016-03-08     -0.215915     -0.550666   -0.183442          -0.075172   
2016-03-09     -0.232154      0.426794   -0.196892           0.154496   
2016-03-10      0.618307      0.001655    0.692773           0.451204   
2016-03-11     -0.356168     -0.414728   -0.287577          -0.360758   
2016-03-14      0.216147      1.901167    0.254998 

Actual  Predicted  Positive Actual signal  \
DataDate                                                  
2018-11-01  0.022489   0.002324                       1   
2018-11-02  0.015558   0.002324                       1   
2018-11-05  0.012907   0.002319                       1   
2018-11-07  0.016846   0.002313                       1   
2018-11-08 -0.002371   0.002309                       0   
2018-11-09 -0.016918   0.002306                       0   
2018-11-12 -0.026763   0.002306                       0   
2018-11-13 -0.036548   0.002308                       0   
2018-11-14  0.061903   0.002310                       1   
2018-11-15 -0.006012   0.002310                       0   
2018-11-16  0.011485   0.002309                       1   
2018-11-19 -0.019260   0.002305                       0   
2018-11-20  0.027288   0.002306                       1   
2018-11-21  0.022854   0.002308                       1   
2018-11-23  0.003375   0.002310                       1   
2018-11-26  0.009341   0.002309                       1   
2018-11-27 -0.014020   0.002306                       0   
2018-11-28  0.020058   0.002305                       1   
2018-11-29  0.004409   0.002306                       1   
2018-11-30 -0.000436   0.002307                       0   
2018-12-03  0.027796   0.002305                       1   
2018-12-04 -0.029406   0.002322                       0   
2018-12-06 -0.047278   0.002323                       0   
2018-12-07 -0.032690   0.002324                       0   
2018-12-10 -0.012046   0.002320                       0   
2018-12-11 -0.052828   0.002313                       0   
2018-12-12  0.013920   0.002311                       1   
2018-12-13 -0.076242   0.002308                       0   
2018-12-14  0.103930   0.002307                       1   
2018-12-17 -0.030544   0.002308                       0   
...              ...        ...                     ...   
2019-12-05  0.064483   0.002306                       1   
2019-12-06 -0.006579   0.002313                       0   
2019-12-09  0.037362   0.002313                       1   
2019-12-10  0.027699   0.002310                       1   
2019-12-11  0.003836   0.002310                       1   
2019-12-12  0.033605   0.002308                       1   
2019-12-13  0.014384   0.002307                       1   
2019-12-16  0.013380   0.002307                       1   
2019-12-17 -0.001300   0.002305                       0   
2019-12-18 -0.036433   0.002304                       0   
2019-12-19  0.028518   0.002310                       1   
2019-12-20  0.029633   0.002310                       1   
2019-12-23  0.019255   0.002311                       1   
2019-12-24  0.038801   0.002309                       1   
2019-12-26  0.049205   0.002309                       1   
2019-12-27 -0.021945   0.002309                       0   
2019-12-30 -0.006627   0.002307                       0   
2020-01-02  0.097689   0.002305                       1   
2020-01-03  0.024883   0.002313                       1   
2020-01-06 -0.036102   0.002314                       0   
2020-01-07 -0.009662   0.002310                       0   
2020-01-08 -0.005823   0.002309                       0   
2020-01-09  0.071890   0.002305                       1   
2020-01-10  0.040863   0.002318                       1   
2020-01-13  0.004635   0.002319                       1   
2020-01-14 -0.012898   0.002315                       0   
2020-01-15 -0.012039   0.002309                       0   
2020-01-16  0.015913   0.002309                       1   
2020-01-17  0.024854   0.002307                       1   
2020-01-21  0.102962   0.002311                       1   

            Negative Actual signal  Positive Predicted signal  \
DataDate                                                        
2018-11-01                       0                          1   
2018-11-02                       0                          1   
2018-11-05                       0  

In [122]:
#stocks['LSTM Predicted Signal'].to_pickle(r'tsla_LSTM.pickle')

In [123]:
#stocks.plot(y = ['Actual Signal', 'LSTM Predicted Signal'], figsize = (20,10), kind = 'bar')

In [124]:
# Calculate cumulative return of model and plot the result
#(1 + (stocks['Actual'] * stocks['LSTM Predicted Signal'])).cumprod().plot(figsize = (20,10))

In [125]:
# Set initial capital allocation
#initial_capital = 100000

# Plot cumulative return of model in terms of capital
#cumulative_return_capital = initial_capital * (1 + (stocks['Actual'] * stocks['LSTM Predicted Signal'])).cumprod()
#cumulative_return_capital.plot(figsize = (20,10))